# NB_Streamer Development Sandbox

This notebook is for experimenting with NB_Streamer functionality during development.

## Setup

In [ ]:
import json
import requests
from datetime import datetime
import sys
import os

# Add src to path for imports
sys.path.append('/app/src')

print("NB_Streamer Development Environment")
print(f"Python version: {sys.version}")
print(f"Current working directory: {os.getcwd()}")

## Test API Connection

In [ ]:
# Test connection to local API
api_base = "http://localhost:8080"

try:
    response = requests.get(f"{api_base}/health")
    print(f"Health check status: {response.status_code}")
    print(f"Response: {response.json()}")
except requests.exceptions.ConnectionError:
    print("API not running. Start it with: python -m uvicorn src.main:app --reload")

## Sample Netbird Event Data

In [ ]:
# Sample Netbird event for testing
sample_event = {
    "ID": "event-123-456",
    "Timestamp": datetime.utcnow().isoformat() + "Z",
    "Message": "Peer connected to network",
    "InitiatorID": "peer-laptop-001",
    "TargetID": "peer-server-001",
    "Meta": {
        "ip": "10.0.0.15",
        "location": "San Francisco",
        "client_version": "0.24.0"
    },
    "Reference": "https://netbird.io/activity/event-123-456"
}

print("Sample Netbird Event:")
print(json.dumps(sample_event, indent=2))

## Test Event Processing

In [ ]:
# Send test event to API
try:
    response = requests.post(
        f"{api_base}/events",
        json=sample_event,
        headers={"Content-Type": "application/json"}
    )
    print(f"Event submission status: {response.status_code}")
    print(f"Response: {response.json()}")
except requests.exceptions.ConnectionError:
    print("API not running. Start it with: python -m uvicorn src.main:app --reload")

## GELF Format Experimentation

Experiment with converting Netbird events to GELF format.

In [ ]:
import time
from datetime import datetime

def netbird_to_gelf(netbird_event):
    """Convert Netbird event to GELF format - experimental implementation."""
    
    # Parse timestamp
    timestamp = datetime.fromisoformat(netbird_event["Timestamp"].replace("Z", "+00:00"))
    unix_timestamp = timestamp.timestamp()
    
    # Create GELF message
    gelf_message = {
        "version": "1.1",
        "host": netbird_event.get("InitiatorID", "unknown"),
        "short_message": netbird_event["Message"],
        "timestamp": unix_timestamp,
        "level": 6,  # Info level
        "full_message": f"{netbird_event['Message']} - Meta: {json.dumps(netbird_event.get('Meta', {}))}",
        
        # Custom Netbird fields with NB_ prefix
        "NB_ID": netbird_event["ID"],
        "NB_TIMESTAMP": netbird_event["Timestamp"],
        "NB_INITIATOR_ID": netbird_event["InitiatorID"],
        "NB_TARGET_ID": netbird_event["TargetID"],
        "NB_REFERENCE": netbird_event["Reference"],
        "NB_META": netbird_event.get("Meta", {})
    }
    
    return gelf_message

# Test conversion
gelf_event = netbird_to_gelf(sample_event)
print("Converted GELF Event:")
print(json.dumps(gelf_event, indent=2))

## Development Notes

Use this section for development notes and experimentation.

In [ ]:
# Development playground - add your experimental code here
print("Ready for development!")
print("Next: Implement Phase 1 features")